<a id='rdw_top'>

# Add extra information from RDW open data

Query to the open data dataset of the RDW.


1. <a href="#rdw_registrations">Registration numbers</a>  
    Apis with registration as key
2. <a href="#rdw_confcodes">Conformity codes</a>  
    Cars get a conformity code when certified.
3. <a href="#rdw_other_apis">Other APIs</a>  
    It may take a while (10 min) to query all conformity codes individually.
4. <a href="#rdw_ovi">Closed data</a>  
    Get closed data from RDW website. This also takes a while because of time out enforced by website. Use config to disable.
5. <a href="#rdw_merge">Merge results</a>  
    Combine all dataframes and save

- - - - 

### User variables


In [1]:
import drz_config
cfg = drz_config.read_config()
DATE = cfg['DATE']
VERBOSE = cfg['VERBOSE']
OPBOD = cfg['OPBOD']
CLOSEDDATA = cfg['CLOSEDDATA']
closed_data_fields = cfg['closed_data_fields']
SKIPSAVE = cfg['SKIPSAVE']

if VERBOSE > 0:
    display(cfg)

{'settings_fn': '../code/assets/drz-auction-settings.ini',
 'DATE': '2021-05',
 'VERBOSE': 1,
 'OPBOD': False,
 'URL': 'http://verkoop.domeinenrz.nl/verkoop_bij_inschrijving_2021-0005',
 'EXTEND_URL': False,
 'CLOSEDDATA': True,
 'closed_data_fields': '*',
 'SKIPSAVE': False}

### Modules and functions

In [2]:
import pandas as pd
import numpy as np
import re 
import os
# to keep api key hidden import this from sub dir
import assets.hidden_api_keys as hidden_api_keys
from time import sleep
import urllib

# base url
apiurl = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json?$$app_token=' + hidden_api_keys.socrata_apptoken + '&'

def get_json_from_api(url,reg,c=0):
    
    '''Get json object from api'''
    
    import time

    c+=1
    try:
        df=pd.read_json(url + 'kenteken=' + reg.replace('-','').upper()).to_dict()
    except:
        if c > 10:
            print(url,reg)
            raise 
        else:
            print('pause 2 sec and try again!')
            time.sleep(2)
            df = get_json_from_api(url,reg,c)
    
    return df
    
# get_json_from_api(apiurl,'61-sf-FG')

### Load auction results

In [3]:
if OPBOD:
    file_name = '../../../python-nb/data/drz-data-opbod-{}.pkl'.format(DATE)
else:
    file_name = '../data/drz-data-{}.pkl'.format(DATE)

if not os.path.isfile(file_name):
    # see if -without price- exists
    NO_PRICE = True
    file_name = file_name.replace('.pkl', '-without-price.pkl')
else:
    NO_PRICE = False

print(file_name)
drz = pd.read_pickle(file_name)

../data/drz-data-2021-05.pkl


### Collect registrations

In [4]:
# see what lots have a Dutch registration (license number).
hasReg = (~drz.Reg.isnull()) & (drz.Reg != 'onbekend') & (drz.Reg != '') & (~drz.LotType.isin([
    'Vaartuig',
    'Jetski',
    'Sloep',
    'Speedboot',
    'Vaartuig (Type onbekend)',
    'Motorvaartuig met opbouw (Pleziervaartuig)',
]))

print('nr. of registrations:',sum(hasReg))

# # make a copy and add info
# rdw = drz.copy()


nr. of registrations: 182


In [5]:
def get_query_url(api_url, keys, token=hidden_api_keys.socrata_apptoken, query_field='kenteken'):
    '''construct query url'''

    import urllib

    # First part
    q = api_url + '?$$app_token=' + token + '&'

    # convert list to string
    id_list = ''.join(["'{}', ".format(k) for k in keys])
    id_list = id_list[0:-2] # remove trailing ', '
    
    # add escaped soql
    soql = '$where=' + urllib.parse.quote(query_field + ' in (' + id_list + ')')
    q += soql
    
    # See if field is available
    accepted_fields = eval(urllib.request.urlopen(api_url).headers.get('X-SODA2-Fields'))
    if query_field not in accepted_fields:
        raise ValueError(f'<{query_field}> not allowed as SODA2 field in {api_url}.\n\tAccepted are: ' + ', '.join(accepted_fields))
    
    return q

Create list of dataframes with different api results

In [6]:
# empty dictionary
rdw_per_reg = dict()

# first element of dict is registrations
key = 'registations'
rdw_per_reg[key] = drz.loc[hasReg,['Reg', 'LotType']].copy() # copy from drz
rdw_per_reg[key]['kenteken'] = rdw_per_reg[key].Reg.apply(lambda r: r.replace('-','').upper())
rdw_per_reg[key].index.name = 'lot_index'
rdw_per_reg[key] = rdw_per_reg[key].reset_index().set_index('kenteken')
display(rdw_per_reg[key].reset_index().set_index(['LotType', 'kenteken']))


lot_index       Reg
LotType                kenteken                        
Bromfiets              DBL09G    2021-05-1800  DBL-09-G
                       D889KD    2021-05-1801  D-889-KD
Motorfiets             92MFZD    2021-05-1802  92-MF-ZD
Bromfiets              DF117B    2021-05-1803  DF-117-B
                       F576TR    2021-05-1804  F-576-TR
...                                       ...       ...
Personenauto           TZ394P    2021-05-8152  TZ-394-P
                       8KTL20    2021-05-8155  8-KTL-20
Aanhangwagen (Caravan) WJ89JD    2021-05-8160  WJ-89-JD
Aanhangwagen           WDVN48    2021-05-8161  WD-VN-48
Bedrijfswagen          V546DR    2021-05-8162  V-546-DR

[182 rows x 2 columns]

<H1><a href="#rdw_top">^</a></H1><a id='rdw_registrations'>

# Main apis

The main api: `api_gekentekende_voertuigen` points to subsequent apis.

In [7]:
# Assess these registrations
regs = rdw_per_reg['registations'].Reg.values
regs = [r.replace('-','').upper() for r in regs]
print(len(regs),'registrations in this set')

# Do main api first to get other possible apis
api_name = 'api_gekentekende_voertuigen'
api_url = 'https://opendata.rdw.nl/resource/m9d7-ebf2.json'

# Query api
key = re.sub('^api_','', api_name) # store with this key
q = get_query_url(api_url,regs)
rdw_per_reg[key] = pd.read_json(q)
rdw_per_reg[key].set_index('kenteken', inplace=True)

print(api_name, end=': ')
display(rdw_per_reg[key])

# Query other available apis
for api_name in [c for c in rdw_per_reg['gekentekende_voertuigen'].columns if c.startswith('api')]:
    print(api_name, end=': ')
    key = re.sub('^api_','',api_name)
    for api_url in rdw_per_reg['gekentekende_voertuigen'][api_name].unique():
        print(api_url)
        # query the web
        q = get_query_url(api_url,regs)
        df0 = pd.read_json(q)
        # name of index
        df0.columns.name = api_name

        # query should return 'kenteken', make it the index
        if df0.shape[0] != 0:
            df0.set_index('kenteken', inplace=True)
            
        # Some apis return multiple values. Pivot around index number ("volgnummer")
        if api_name == 'api_gekentekende_voertuigen_assen':
            df0 = pd.pivot(df0, columns='as_nummer')

        elif api_name == 'api_gekentekende_voertuigen_brandstof':
            df0 = pd.pivot(df0, columns='brandstof_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie':
            df0 = pd.pivot(df0, columns='carrosserie_volgnummer')

        elif api_name == 'api_gekentekende_voertuigen_carrosserie_specifiek':
            df0 = pd.pivot(df0, columns='carrosserie_voertuig_nummer_code_volgnummer')

        # squeeze multi index
        one_level = [
            re.sub('^api_gekentekende_voertuigen_','',api_name) + '_' + '_'.join(
                [str(c) if type(c)==int else c for c in l]
            ) for l in df0.columns
        ]
        df0.columns = one_level

        # add to list
        display(df0.tail(3))
        print(df0.shape)
        rdw_per_reg[key]=df0

# Merge dataframes from different apis
df_regs = pd.concat(rdw_per_reg.values(), axis='columns', sort=False)
# add timestamp
df_regs['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')
# set lot id as index
df_regs.index.name = 'kenteken'
df_regs = df_regs.reset_index().set_index('lot_index')
display(df_regs)

182 registrations in this set
api_gekentekende_voertuigen: 

,voertuigsoort,merk,handelsbenaming,datum_tenaamstelling,bruto_bpm,inrichting,aantal_zitplaatsen,eerste_kleur,tweede_kleur,aantal_cilinders,...,maximum_massa_trekken_ongeremd,maximum_trekken_massa_geremd,zuinigheidslabel,plaats_chassisnummer,jaar_laatste_registratie_tellerstand,maximale_constructiesnelheid_brom_snorfiets,vermogen_brom_snorfiets,laadvermogen,aanhangwagen_autonoom_geremd,aanhangwagen_middenas_geremd
kenteken,,,,,,,,,,,,,,,,,,,,,
01MNFV,Motorfiets,KAWASAKI,Z H2,20210331,2589.0,Niet geregistreerd,2.0,N.v.t.,N.v.t.,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01NPL3,Personenauto,FIAT,FIAT PUNTO,20210401,NaN,MPV,5.0,ZWART,Niet geregistreerd,4.0,...,400.0,1000.0,A,r. op bodemplaat by voorzitting,2021.0,NaN,NaN,NaN,NaN,NaN
02SXKS,Personenauto,VOLKSWAGEN,POLO,20210219,3492.0,hatchback,5.0,ZWART,Niet geregistreerd,4.0,...,540.0,1000.0,D,op r. schroefveerkoker onder motorkap,2021.0,NaN,NaN,NaN,NaN,NaN
04RZJB,Personenauto,KIA,RIO,20210408,2295.0,MPV,5.0,GRIJS,Niet geregistreerd,4.0,...,453.0,1100.0,B,tegen dwarsbalk by r. voorzitting,2021.0,NaN,NaN,NaN,NaN,NaN
04TJVP,Personenauto,RENAULT,MEGANE,20210315,7578.0,cabriolet,4.0,GRIJS,Niet geregistreerd,4.0,...,650.0,1200.0,C,r. in motorruimte op steun,2021.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XX155R,Personenauto,MERCEDES-BENZ,GLA 180,20210413,4379.0,stationwagen,5.0,ZWART,Niet geregistreerd,4.0,...,715.0,1200.0,D,NaN,2021.0,NaN,NaN,NaN,NaN,NaN
ZH659L,Personenauto,BMW,1ER REIHE,20210401,470.0,stationwagen,5.0,GRIJS,Niet geregistreerd,4.0,...,690.0,1200.0,C,NaN,2021.0,NaN,NaN,NaN,NaN,NaN
ZS658L,Personenauto,VOLVO,XC90,20210406,14620.0,MPV,7.0,GRIJS,Niet geregistreerd,4.0,...,750.0,2400.0,E,NaN,2021.0,NaN,NaN,NaN,NaN,NaN


api_gekentekende_voertuigen_assen: https://opendata.rdw.nl/resource/3huj-srit.json


,assen_aantal_assen_1,assen_aantal_assen_2,assen_spoorbreedte_1,assen_spoorbreedte_2,assen_wettelijk_toegestane_maximum_aslast_1,assen_wettelijk_toegestane_maximum_aslast_2,assen_technisch_toegestane_maximum_aslast_1,assen_technisch_toegestane_maximum_aslast_2,assen_plaatscode_as_1,assen_plaatscode_as_2,assen_hefas_1,assen_hefas_2,assen_aangedreven_as_1,assen_aangedreven_as_2
kenteken,,,,,,,,,,,,,,
ZS658L,2.0,2.0,168.0,168.0,1310.0,1490.0,1310.0,1490.0,V,A,NaN,NaN,J,J
ZS828G,2.0,2.0,158.0,156.0,1050.0,1115.0,1050.0,1115.0,V,A,NaN,NaN,N,J
ZT481S,2.0,2.0,155.0,152.0,1070.0,900.0,1070.0,900.0,V,A,NaN,NaN,J,N


(120, 14)
api_gekentekende_voertuigen_brandstof: https://opendata.rdw.nl/resource/8ys7-d773.json


...

,carrosserie_specifiek_carrosserie_volgnummer_1,carrosserie_specifiek_carrosseriecode_1,carrosserie_specifiek_carrosserie_voertuig_nummer_europese_omschrijving_1
kenteken,,,
V546DR,1,3,Gesloten opbouw
VJ359F,1,3,Gesloten opbouw
WDVN48,1,3,Gesloten opbouw


(4, 3)
api_gekentekende_voertuigen_voertuigklasse: https://opendata.rdw.nl/resource/kmfi-hrps.json


""


(0, 0)


,kenteken,Reg,LotType,voertuigsoort,merk,handelsbenaming,datum_tenaamstelling,bruto_bpm,inrichting,aantal_zitplaatsen,...,brandstof_netto_max_vermogen_elektrisch_1,brandstof_netto_max_vermogen_elektrisch_2,brandstof_klasse_hybride_elektrisch_voertuig_1,brandstof_klasse_hybride_elektrisch_voertuig_2,carrosserie_carrosserietype_1,carrosserie_type_carrosserie_europese_omschrijving_1,carrosserie_specifiek_carrosserie_volgnummer_1,carrosserie_specifiek_carrosseriecode_1,carrosserie_specifiek_carrosserie_voertuig_nummer_europese_omschrijving_1,TimeStamp
lot_index,,,,,,,,,,,,,,,,,,,,,
2021-05-1800,DBL09G,DBL-09-G,Bromfiets,Bromfiets,PIAGGIO,VESPA SPRINT,20210324,NaN,N.v.t.,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210507
2021-05-1801,D889KD,D-889-KD,Bromfiets,Bromfiets,SYM,HU05W,20210324,NaN,N.v.t.,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210507
2021-05-1802,92MFZD,92-MF-ZD,Motorfiets,Motorfiets,PIAGGIO,N/A,20210316,702.0,Niet geregistreerd,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210507
2021-05-1803,DF117B,DF-117-B,Bromfiets,Bromfiets,PIAGGIO,C38,20210324,NaN,N.v.t.,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210507
2021-05-1804,F576TR,F-576-TR,Bromfiets,Bromfiets,SYM,MIO 50,20210324,NaN,N.v.t.,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-8152,TZ394P,TZ-394-P,Personenauto,Personenauto,OPEL,INSIGNIA GRAND SPORT,20201229,944.0,hatchback,5.0,...,NaN,NaN,NaN,NaN,AB,Hatchback,NaN,NaN,NaN,20210507
2021-05-8155,8KTL20,8-KTL-20,Personenauto,Personenauto,VOLKSWAGEN,PASSAT,20190930,4573.0,stationwagen,5.0,...,NaN,NaN,NaN,NaN,AC,Stationwagen,NaN,NaN,NaN,20210507
2021-05-8160,WJ89JD,WJ-89-JD,Aanhangwagen (Caravan),Aanhangwagen,HOME-CAR,36,20210304,NaN,caravan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210507


<H1><a href="#rdw_top">^</a></H1><a id='rdw_confcodes'>

# Conformity codes

In [8]:
# empty dict
rdw_per_confcode = dict()
# Conformity codes have sub-divisions. Four fields make a super key
key = 'conformity_codes'
rdw_per_confcode[key] = rdw_per_reg['gekentekende_voertuigen'][[
    'typegoedkeuringsnummer', 
    'uitvoering', 
    'variant', 
    'volgnummer_wijziging_eu_typegoedkeuring'
]].dropna().drop_duplicates()
rdw_per_confcode[key].reset_index(drop=True, inplace=True)
print(len(rdw_per_confcode[key]),'conformity codes in this set')

display(rdw_per_confcode[key])
print('count unique')
display(\
    rdw_per_reg['gekentekende_voertuigen'].loc[:, rdw_per_confcode[key].columns]\
    .reset_index()\
    .groupby([rdw_per_confcode[key].columns[0]])\
    .nunique()\
    .replace(1,np.NaN)\
    .dropna(how='all')\
    .fillna(1)\
    .astype(int)\
    .sort_values(by='kenteken', ascending=False)
)

153 conformity codes in this set


,typegoedkeuringsnummer,uitvoering,variant,volgnummer_wijziging_eu_typegoedkeuring
0,e4*168/2013*00117*00,E,K1,0.0
1,e3*2001/116*0217*19,48,AXY1A,0.0
2,e1*2001/116*0174*16,FM5FM52T0081N1K0GG,ABSBKYX01,0.0
3,e4*2001/116*0093*02,511A,M111,0.0
4,e2*98/14*0272*26,EM1R0H,EM1R,0.0
...,...,...,...,...
148,e1*2001/116*0470*20,OZAA15A1,274GQ2,0.0
149,e1*2007/46*0283*04,H2,1C11,0.0
150,e4*2007/46*0929*06,LFA7BC??,LFA7,0.0
151,e1*2001/116*0431*38,NZAA0574,R00XP0,0.0


count unique


,kenteken,uitvoering,variant,volgnummer_wijziging_eu_typegoedkeuring
typegoedkeuringsnummer,,,,
e11*2002/24*1744*02,5,1,1,1
e4*2002/24*2691*09,4,1,1,1
e9*168/2013*11160*00,4,1,1,1
e4*2002/24*0361*02,3,1,2,1
e4*2002/24*0403*05,3,1,1,1
e4*2002/24*0432*02,3,2,1,1
e1*2001/116*0174*16,2,2,2,1
e1*2001/116*0430*12,2,2,2,1
e1*2001/116*0510*11,2,2,2,1


In [9]:
# do conformity api and again get other possible apis
api_name = 'api_eeg_voertuigtypegoedkeuring'
api_url = 'https://opendata.rdw.nl/resource/55kv-xf7m.json'

# Query api
key = re.sub('^api_','', api_name) # store with this key
q = get_query_url(
    api_url, 
    rdw_per_confcode['conformity_codes'].typegoedkeuringsnummer.unique(), # Use long (year with century) version of conformity code
    query_field='typegoedkeuringsnummer'
)
rdw_per_confcode[key] = pd.read_json(q)
# more than one conformity code?
assert not (rdw_per_confcode[key].groupby('typegoedkeuringsnummer')['typegoedkeuringsnummer'].count() > 1).any()
rdw_per_confcode[key].set_index('typegoedkeuringsnummer', inplace=True)

# add slightly different keys (year has no century)
rdw_per_confcode['conformity_codes'] = rdw_per_confcode['conformity_codes'].merge(
    rdw_per_confcode['eeg_voertuigtypegoedkeuring'].eu_type_goedkeuringssleutel, 
    how='left', 
    left_on='typegoedkeuringsnummer', 
    right_index=True
)

print(api_name, end=': ')
display(rdw_per_confcode[key])




api_eeg_voertuigtypegoedkeuring: 

,eu_type_goedkeuringssleutel,landcode_eeg_typegoedkeuring,richtlijn_nr_laatste_wijziging,eeg_basis_goedkeuringsnummer,eeg_uitbreiding_goedkeuringsnummer,eeg_typegoedkeuringsdatum,europese_typegoedkeurings_registratie_datum,eeg_ece_voertuig_categorie_bij_type,fabrikant,europese_typegoedkeuring_status,...,api_carrosserie_uitvoering_klasse,api_carrosserie_uitvoering_nummerieke_code,api_handelsbenaming_uitvoering,api_merk_uitvoering_toegestaan,api_motor_uitvoering,api_motor_uitvoering_brandstof,api_plaatsaanduiding_uitvoering,api_subcategorie_uitvoering,api_uitvoeringverbruik_per_uitgave,api_versnellingsbak_uitvoering
typegoedkeuringsnummer,,,,,,,,,,,,,,,,,,,,,
e11*168/2013*00289*03,e11*168/13*00289*03,e11,168/13,289.0,3,20180525,20180525,L1,PIAGGIO & C. S.P.A.,AT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
e11*168/2013*00289*04,e11*168/13*00289*04,e11,168/13,289.0,4,20180619,20180619,L1,PIAGGIO & C. S.P.A.,AT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
e11*168/2013*00290*00,e11*168/13*00290*00,e11,168/13,290.0,0,20170928,20170928,L1,"ZHEJIANG SHANQI TIANYING VEHICLE INDUSTRY CO.,...",BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
e11*168/2013*00290*02,e11*168/13*00290*02,e11,168/13,290.0,2,20180308,20180308,L1,"ZHEJIANG SHANQI TIANYING VEHICLE INDUSTRY CO.,...",BT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
e11*2001/116*0203*04,e11*01/116*0203*04,e11,01/116,203.0,4,20050907,20050907,M1,MAZDA MOTOR CORPORATION,AT,...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...,https://opendata.rdw.nl/Voertuigen/Open-Data-R...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e9*2001/116*0052*08,e9*01/116*0052*08,e9,01/116,52.0,8,20060721,20060721,M1,SEAT S.A.,BT,...,https://opendata.rdw.nl/Voertuigen/Open-Da

<H1><a href="#rdw_top">^</a></H1><a id='rdw_other_apis'>

# Other apis

In [10]:
# query other available apis
api_list = [c for c in rdw_per_confcode['eeg_voertuigtypegoedkeuring'].columns if c.startswith('api')]
for api_name in api_list:
    print(api_name, end=''.join(['|' if api_name==list_element else '.'  for list_element in api_list]))
    key = re.sub('^api_','',api_name)
    for api_url in rdw_per_confcode['eeg_voertuigtypegoedkeuring'][api_name].unique():
        
        # reformat url
        M=re.search('https://opendata.rdw.nl/.*/([a-z0-9]{4}-[a-z0-9]{4})$', api_url)
        api_url = 'https://opendata.rdw.nl/resource/{}.json'.format(M[1])
        print(api_url)
       
        # query the web
        # unfortunately this needs to be done one by one, because conformity code is not unique                   
        df0 = pd.DataFrame()
        for ix, row in rdw_per_confcode['conformity_codes'].iterrows():
            q = api_url
            q += '?$$app_token=' + hidden_api_keys.socrata_apptoken 
            q += '&{}=\'{}\''.format('eu_type_goedkeuringssleutel', urllib.parse.quote(row.eu_type_goedkeuringssleutel))
            q += '&{}=\'{}\''.format('eeg_uitvoeringscode', urllib.parse.quote(row.uitvoering))
            q += '&{}=\'{}\''.format('eeg_variantcode', urllib.parse.quote(row.variant))
            q += '&{}={:.0f}'.format('uitvoering_wijzigingsnummer', row.volgnummer_wijziging_eu_typegoedkeuring)
                
            n_try = 0
            OK = False
            while (n_try < 10) & (not OK):
                try:
                    res = pd.read_json(q)
                    OK = True
                    n_try = 0
                except:
                    n_try +=1
                    sleep(10)
                if n_try == 10:
                    raise

            if len(res) == 0:
                continue

            # matching data type with 'codes' for merging
            res.eu_type_goedkeuringssleutel = res.eu_type_goedkeuringssleutel.astype(str)
            res.eeg_uitvoeringscode = res.eeg_uitvoeringscode.astype(str)
            res.eeg_variantcode = res.eeg_variantcode.astype(str)
            res.uitvoering_wijzigingsnummer = res.uitvoering_wijzigingsnummer.astype(float)
            res.set_index([
                'eu_type_goedkeuringssleutel', 
                'eeg_uitvoeringscode', 
                'eeg_variantcode', 
                'uitvoering_wijzigingsnummer'
            ], inplace=True)
                
                
            # Api specific modifiers
            if api_name == 'api_as_gegevens_eeg_uitvoering':               
                piv = res.pivot(columns='asnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)                
            elif api_name == 'api_handelsbenaming_uitvoering':                
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_carrosserie_uitvoering':                
                piv = res.pivot(columns='carrosserie_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_plaatsaanduiding_uitvoering':
                piv = res.pivot(columns='plaats_aanduiding_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)            
            elif api_name == 'api_carrosserie_uitvoering_nummerieke_code':
                piv = res.pivot(columns='carrosserie_uitvoering_numeriek_volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)            
            elif api_name == 'api_uitvoeringverbruik_per_uitgave':
                piv = res.pivot(columns='uitvgavenummer_verbruikboek')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_motor_uitvoering':
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_versnellingsbak_uitvoering':
                piv = res.pivot(columns='volgnummer')
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)
            elif api_name == 'api_motor_uitvoering_brandstof':
                piv = pd.pivot_table(res, index=res.index.names, columns=['volgnummer', 'brandstof_volgnummer'])
                one_level = [
                            re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)) + '_' + '_'.join(
                                [str(c) if type(c)==int else c for c in l]
                            ) for l in piv.columns
                        ]
                piv.columns = one_level
                df0 = pd.concat([df0, piv], sort=False)                
            elif api_name == 'api_merk_uitvoering_toegestaan':
                piv = pd.pivot_table(res, index=res.index.names, values=res.columns, aggfunc=list)
                df0 = pd.concat([df0, piv], sort=False)
            else:
                assert res.shape[0] == 1 # Nothing to pivot on
                columns = ['{}_{}'.format(re.sub('^api_','', re.sub('_eeg_uitvoering$','',api_name)), c) for c in res.columns] 
                res.columns = columns
                df0 = pd.concat([df0, res], sort=False)
                
            print('.', end='')
        if len(df0)==0:
            print('No results for this api')
            continue


        # add to dict
        rdw_per_confcode[key] = df0
        display(rdw_per_confcode[key].tail())
        print(rdw_per_confcode[key].shape)


api_basisgegevens_eeg_uitvoering|............https://opendata.rdw.nl/resource/wx3j-69ie.json
.........................................................................................................................................................

,,,,basisgegevens_uitvoering_registratie_datum,basisgegevens_begindatum_uitvoering,basisgegevens_aantal_wielen,basisgegevens_links_rechts_rijdend,basisgegevens_status_voertiug_kentekening,basisgegevens_wielbasis_ondergrens,basisgegevens_wielbais_bovengrens,basisgegevens_aantal_zitplaatsen_ondergrens,basisgegevens_aantal_zitplaatsen_bovengrens,basisgegevens_massa_bedrijfsklaar_toestand_ondergrens,...,basisgegevens_eur_codering_carrosserietype,basisgegevens_handelsbenaming,basisgegevens_einddatum_uitvoering,basisgegevens_massa_leeg_voertuig_ondergrens,basisgegevens_massa_leeg_voertuig_bovengrens,basisgegevens_inhoud_brandstoftank,basisgegevens_eeg_uitvoering_cat_toevoeging,basisgegevens_kant_van_het_stuur,basisgegevens_techn_max_massa_autonoom_ondergrens,basisgegevens_techn_max_massa_autonoom_bovengrens
eu_type_goedkeuringssleutel,eeg_uitvoeringscode,eeg_variantcode,uitvoering_wijzigingsnummer,,,,,,,,,,,,,,,,,,,,,
e1*01/116*0470*20,OZAA15A1,274GQ2,0.0,20180111,20180111,4,NaN,C,2699,2699,5.0,5.0,1435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e1*07/46*0283*04,H2,1C11,0.0,20110518,20110518,4,R,C,2690,2690,5.0,5.0,1395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e4*07/46*0929*06,LFA7BC??,LFA7,0.0,20171027,20171027,4,B,C,2984,2984,7.0,7.0,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN
e1*01/116*0431*38,NZAA0574,R00XP0,0.0,20151120,20151120,4,NaN,C,2840,2840,5.0,5.0,1570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e1*07/46*0623*36,FD7FD7GC003S7CSON1ML1CVR2NA,GAC4DKTBX0,0.0,20181122,20181122,4,B,C,2626,2626,5.0,5.0,1445,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(153, 49)
api_as_gegevens_eeg_uitvoering.|...........https://opendata.rdw.nl/resource/ahsi-8uyu.json
.............................................................................................................................

,,,,as_gegevens_aangedreven_as_1,as_gegevens_aangedreven_as_2,as_gegevens_dubbele_montage_1,as_gegevens_dubbele_montage_2,as_gegevens_techn_max_last_as_ondergrens_1,as_gegevens_techn_max_last_as_ondergrens_2,as_gegevens_techn_max_last_as_bovengrens_1,as_gegevens_techn_max_last_as_bovengrens_2,as_gegevens_spoorbreedte_ondergrens_1,as_gegevens_spoorbreedte_ondergrens_2,...,as_gegevens_hefas_1,as_gegevens_hefas_2,as_gegevens_plaats_as_code_1,as_gegevens_plaats_as_code_2,as_gegevens_luchtvering_1,as_gegevens_luchtvering_2,as_gegevens_gelijkwaardig_aan_luchtvering_1,as_gegevens_gelijkwaardig_aan_luchtvering_2,as_gegevens_gestuurde_as_indicator_1,as_gegevens_gestuurde_as_indicator_2
eu_type_goedkeuringssleutel,eeg_uitvoeringscode,eeg_variantcode,uitvoering_wijzigingsnummer,,,,,,,,,,,,,,,,,,,,,
e1*01/116*0470*20,OZAA15A1,274GQ2,0.0,J,N,NaN,NaN,1020.0,920.0,1020,920,1540.0,1544.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e1*07/46*0283*04,H2,1C11,0.0,N,J,NaN,NaN,885.0,1055.0,885,1055,1511.0,1529.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e4*07/46*0929*06,LFA7BC??,LFA7,0.0,J,J,NaN,NaN,1310.0,1490.0,1310,1490,1665.0,1667.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e1*01/116*0431*38,NZAA0574,R00XP0,0.0,N,J,NaN,NaN,1050.0,1115.0,1050,1115,1561.0,1523.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e1*07/46*0623*36,FD7FD7GC003S7CSON1ML1CVR2NA,GAC4DKTBX0,0.0,J,N,NaN,NaN,1070.0,900.0,1070,900,1533.0,1503.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(125, 32)
api_carrosserie_uitvoering..|..........https://opendata.rdw.nl/resource/w2qp-idms.json
...........................................................................................................

...

(1, 2)
api_uitvoeringverbruik_per_uitgave...........|.https://opendata.rdw.nl/resource/2822-t8sx.json
........................................................................................................

,,,,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_10,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_11,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_12,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_13,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_14,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_15,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_16,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_17,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_18,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_5,...,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_7,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_8,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_9,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_19,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_1,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_2,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_3,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_4,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_20,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_21
eu_type_goedkeuringssleutel,eeg_uitvoeringscode,eeg_variantcode,uitvoering_wijzigingsnummer,,,,,,,,,,,,,,,,,,,,,
e1*01/116*0470*20,OZAA15A1,274GQ2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,...,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN
e1*07/46*0283*04,H2,1C11,0.0,NaN,B,C,C,E,E,F,F,F,NaN,...,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN
e4*07/46*0929*06,LFA7BC??,LFA7,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,E,NaN,...,NaN,NaN,NaN,E,NaN,NaN,NaN,NaN,NaN,NaN
e1*01/116*0431*38,NZAA0574,R00XP0,0.0,NaN,NaN,NaN,NaN,NaN,D,E,E,E,NaN,...,NaN,NaN,NaN,E,NaN,NaN,NaN,NaN,NaN,NaN
e1*07/46*0623*36,FD7FD7GC003S7CSON1ML1CVR2NA,GAC4DKTBX0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(104, 21)
api_versnellingsbak_uitvoering............|https://opendata.rdw.nl/resource/r7cw-67gs.json
.............................................................................................................................

,,,,versnellingsbak_uitvoering_type_versnellingsbak_1,versnellingsbak_uitvoering_aantal_versnellingen_ondergrens_1,versnellingsbak_uitvoering_aantal_versnellingen_bovengrens_1
eu_type_goedkeuringssleutel,eeg_uitvoeringscode,eeg_variantcode,uitvoering_wijzigingsnummer,,,
e1*01/116*0470*20,OZAA15A1,274GQ2,0.0,A,7.0,7.0
e1*07/46*0283*04,H2,1C11,0.0,H,6.0,6.0
e4*07/46*0929*06,LFA7BC??,LFA7,0.0,A,8.0,8.0
e1*01/116*0431*38,NZAA0574,R00XP0,0.0,A,7.0,7.0
e1*07/46*0623*36,FD7FD7GC003S7CSON1ML1CVR2NA,GAC4DKTBX0,0.0,A,7.0,7.0


(125, 3)


<H1><a href="#rdw_top">^</a></H1><a id='rdw_ovi'>

# Closed data from rdw (OVI)
Optionally get data from rdw website

In [11]:
def get_closed_data(reg, fields = '*'):
    web_url = f'https://ovi.rdw.nl/?kenteken={reg}'

    import requests
    import codecs
    from lxml import html, etree
    import re
    from time import sleep
    
    def _all_fields(tree):
        elements = tree.xpath('//*[@class="ui-block-d border ovigrid"]') + tree.xpath('//*/div[@class="ui-block-d ovigrid"]')
        return [el.attrib['id'] for el in elements if 'id' in el.keys()]
    
    # get page
    page = requests.get(web_url)
    DecodeType = re.findall('charset=(.*)$', page.headers["Content-type"])[0]
    htmlstring = codecs.decode(page.content, DecodeType)
    # Convert string to tree object
    tree = html.fromstring(htmlstring)

    # analyze results
    ERROR = len(tree.xpath('//*[@action="FoutPagina.aspx?error=ErrorBlocked"]')) == 1
    warn_text = tree.xpath('//*[@class="warning"]/*/text()')
    WARNING_NA = (len(warn_text) > 0) and (warn_text[0].startswith('Er zijn geen gegevens gevonden voor het ingevulde kenteken '))

    # return result
    if ERROR:
        print('X', end='')
        return False
    elif WARNING_NA:
        if VERBOSE > 1: print(warn_text[0])
        print('x', end='')
        return None
    else:
        if fields == '*':
            fields = _all_fields(tree)
        out = dict()
        for fld in fields:
            txt = tree.xpath(f'//*[@id="{fld}"]/text()')
            if len(txt) != 1:
                # No result with this field. Just skip
                if VERBOSE > 1: print(fld)
                print('-', end='')
            else:
                print('.', end='')
                out[fld] = str(txt[0])
        # No result with these fields. Probably limit request
        if all([v is None for v in out.values()]):
            raise AssertionError(f'No result with fields: {", ".join(fields)}.')


    return out
    

In [12]:
if CLOSEDDATA:
    data = {k: False for k in df_regs.kenteken}
    if VERBOSE > 1: print('X: error\n.: info retrieved\n-: N.A.')
    DO_LOOP = True
    c = 0 #counter
    max_loop = 2 * len(data) / 30 # approx nr of succesful retrievals is 30
    while DO_LOOP or c > max_loop:
        # pause
        if c > 0:
            if VERBOSE > 1: print(f' time out (60s) for next iteration', end='')
            sleep(60)
            if VERBOSE > 1: print(f': {c:2.0f}/{max_loop:2.0f}')
        c+=1 

        # has no value or field has no value
        to_do_regs = [k for k,v in data.items() if v == False]
        if VERBOSE > 0: print(f'\nto do:{len(to_do_regs):3.0f}', end='')
        data.update({reg: get_closed_data(reg, closed_data_fields) for reg in to_do_regs})
        DO_LOOP = any([v==False for v in data.values()])

    # drop None
    data = {k:v for k,v in data.items() if v is not None}    
    rdw_closed = pd.DataFrame.from_dict(data=data, orient='index')
else:
    rdw_closed = None

rdw_closed


to do:182...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................-..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

,Merk,Type,Variant,Uitvoering,Handelsbenaming,Typegoedkeuring,PlaatsChassisOmschrijving,Eigenaren,EersteToelatingsdatum,DatumAanvangTenaamstelling,...,MaximumMassaAutonoomGeremd,MaximumMassaOpleggerGeremd,Breedte,AantalDeuren,AantalStaanplaatsen,AfwijkendeMaxSnelheid,Laadvermogen,Lengte,SubCatOmschrijving,AfstandHartkoppelingTotAchterzijdeVrtg
DBL09G,PIAGGIO,C53,3,01,VESPA SPRINT,e11*2002/24*1744*02,"Rechts, rechter frame onder zadel",2 / 0,01-05-2015,24-03-2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D889KD,SYM,Niet geregistreerd,IIC,N/A,HU05W,e4*2002/24*0361*02,"Rechts, rechter tegen framebuis onder balhoofd",8 / 0,27-10-2009,24-03-2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92MFZD,PIAGGIO,M45,2,02,N/A,e3*2002/24*0306*10,"Rechts, rechter tegen dwarsbalk onder zadel",4 / 0,01-07-2015,16-03-2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DF117B,PIAGGIO,Niet geregistreerd,3,00,C38,e3*92/61*0125*06,"Rechts, rechter tegen frameplaat onder zadel",7 / 0,05-09-2006,24-03-2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F576TR,SYM,HU05W,IIC,N/A,MIO 50,e4*2002/24*0361*03,"Rechts, rechter tegen framebuis onder balhoofd",2 / 0,03-07-2014,24-03-2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TZ394P,OPEL,Z-B,CA0HMBB12,BA1M2FAFJ5,INSIGNIA GRAND SPORT,e8*2007/46*0264*01,Niet geregistreerd,2 / 1,20-07-2017,29-12-2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8KTL20,VOLKSWAGEN,3C,ACCFGCX0,FD6FD62E018STP17MQSNVR2M,PASSAT,e1*2001/116*0307*33,"Op rechts, rechter voorwielscherm onder motorkap",2 / 1,10-01-2013,30-09-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WJ89JD,HOME-CAR,Niet geregistreerd,Niet geregistreerd,Niet geregistreerd,36,Niet geregistreerd,"Rechts, rechter op trekdriehoek 060 cm Achter ...",1 / 0,15-09-1992,04-03-2021,...,NaN,NaN,210 cm,NaN,NaN,NaN,249 kg,0 cm,NaN,537 cm
WDVN48,SARIS,Niet geregistreerd,Niet geregistreerd,Niet geregistreerd,PK,Niet geregistreerd,"Tegen rechts, rechter balk 190 cm voor hart as...",1 / 0,24-02-2009,22-12-2020,...,NaN,NaN,195 cm,NaN,NaN,NaN,1920 kg,0 cm,NaN,565 cm


<H1><a href="#rdw_top">^</a></H1><a id='rdw_merge'>

# Merge datasets
- Merge dataframes from conformity codes apis
- Merge with registration results
- Merge with auction results

In [13]:
# first merge first two results
tmp = rdw_per_confcode['conformity_codes'].merge(rdw_per_confcode['eeg_voertuigtypegoedkeuring'], how='left', 
                                              left_on='typegoedkeuringsnummer',
                                              right_index=True
                                             )

tmp.rename(columns={
    'eu_type_goedkeuringssleutel_x': 'eu_type_goedkeuringssleutel',
    'uitvoering': 'eeg_uitvoeringscode',
    'variant': 'eeg_variantcode',
    'volgnummer_wijziging_eu_typegoedkeuring': 'uitvoering_wijzigingsnummer',
}, inplace=True)
tmp.set_index(['eu_type_goedkeuringssleutel', 'eeg_uitvoeringscode', 'eeg_variantcode', 'uitvoering_wijzigingsnummer'], inplace=True)

# merge with subsequent api results
df_confcodes = pd.concat([tmp] + list(rdw_per_confcode.values())[2:], axis='columns', sort=False)
# add timestamp
df_confcodes['TimeStamp'] = pd.to_datetime('now').strftime('%Y%m%d')

display(df_confcodes.tail())
print(df_confcodes.shape)

typegoedkeuringsnummer  \
eu_type_goedkeuringssleutel eeg_uitvoeringscode eeg_variantcode uitvoering_wijzigingsnummer                          
e9*168/13*09805*00          00                  2               0.0                           e9*168/2013*11160*00   
e9*168/13*09889*00          01                  00              0.0                           e9*168/2013*11077*00   
                            1                   0               0.0                                            NaN   
e9*168/13*11171*02          0                   1               0.0                                            NaN   
                            00                  01              0.0                           e9*168/2013*11171*02   

                                                                                            eu_type_goedkeuringssleutel_y  \
eu_type_goedkeuringssleutel eeg_uitvoeringscode eeg_variantcode uitvoering_wijzigingsnummer                                 
e9*168/13*09805*00          00                  2               0.0                                    e9*168/13*09805*00   
e9*168/13*09889*00          01                  00              0.0                                    e9*168/13*09889*00   
                            1                   0               0.0                                                   NaN   
e9*168/13*11171*02          0                   1               0.0                                                   NaN   
                            00                  01              0.0                                    e9*168/13*11171*02   

                                                                                            landcode_eeg_typegoedkeuring  \
eu_type_goedkeuringssleutel eeg_uitvoeringscode eeg_variantcode uitvoering_wijzigingsnummer                                
e9*168/13*09805*00          00                  2               0.0                                                   e9   
e9*168/13*09889*00          01                  00              0.0                                                   e9   
                            1                   0               0.0                                                  NaN   
e9*168/13*11171*02          0                   1               0.0                                                  NaN   
                            00                  01              0.0                                                   e9   

                                                                                            richtlijn_nr_laatste_wijziging  \
eu_type_goedkeuringssleutel eeg_uitvoeringscode eeg_variantcode uitvoering_wijzigingsnummer                                  
e9*168/13*09805*00          00                  2               0.0                                                 168/13   
e9*168/13*09889*00          01                  00              0.0                                                 168/13   
                            1                   0               0.0                                                    NaN   
e9*168/13*11171*02          0                   1               0.0                                                    NaN   
                            00                  01              0.0                                                 168/13   

                                                                                             eeg_basis_goedkeuringsnummer  \
eu_type_goedkeuringssleutel eeg_uitvoeringscode eeg_variantcode uitvoering_wijzigingsnummer                                 
e9*168/13*09805*00          00                  2               0.0                                                9805.0   
e9*168/13*09889*00          01                  00              0.0                                                9889.0   
                            1                   0               0.0                                                   NaN   
e9*168/13*11

(179, 444)


In [14]:
# Merge confirmation codes with registrations
df_regs.index.name = 'lot_index'
df = df_regs.reset_index().merge(df_confcodes.reset_index(), how='left',
                   left_on=['typegoedkeuringsnummer', 'uitvoering', 'variant', 'volgnummer_wijziging_eu_typegoedkeuring'],
                   right_on=['typegoedkeuringsnummer', 'eeg_uitvoeringscode', 'eeg_variantcode', 'uitvoering_wijzigingsnummer'],
).set_index('lot_index')
display(df.tail(10))
print(df.shape)
assert all(df.columns.value_counts() == 1)

,kenteken,Reg,LotType,voertuigsoort,merk,handelsbenaming,datum_tenaamstelling,bruto_bpm,inrichting,aantal_zitplaatsen,...,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_1,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_2,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_3,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_4,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_20,uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_21,versnellingsbak_uitvoering_type_versnellingsbak_1,versnellingsbak_uitvoering_aantal_versnellingen_ondergrens_1,versnellingsbak_uitvoering_aantal_versnellingen_bovengrens_1,TimeStamp_y
lot_index,,,,,,,,,,,,,,,,,,,,,
2021-05-8143,WD30YR,WD-30-YR,Aanhangwagen,Aanhangwagen,ANSSEMS,A-S 2000 V V,20210414,NaN,open wagen,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-8146,H147KN,H-147-KN,Personenauto,Personenauto,JAGUAR,JAGUAR XF SPORTBRAKE,20210324,1462.0,stationwagen,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,A,8.0,8.0,20210507
2021-05-8147,JB863V,JB-863-V,Personenauto,Personenauto,MERCEDES-BENZ,CITAN,20210301,8618.0,stationwagen,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,H,6.0,6.0,20210507
2021-05-8148,04TJVP,04-TJ-VP,Personenauto,Personenauto,RENAULT,MEGANE,20210315,7578.0,cabriolet,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,H,5.0,5.0,20210507
2021-05-8149,98RBXG,98-RB-XG,Personenauto,Personenauto,MERCEDES-BENZ,CLK 200 KOMPRESSOR,20210208,17128.0,coupe,4.0,...,NaN,NaN,NaN,D,NaN,NaN,A,5.0,5.0,20210507
2021-05-8152,TZ394P,TZ-394-P,Personenauto,Personenauto,OPEL,INSIGNIA GRAND SPORT,20201229,944.0,hatchback,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,H,6.0,6.0,20210507
2021-05-8155,8KTL20,8-KTL-20,Personenauto,Personenauto,VOLKSWAGEN,PASSAT,20190930,4573.0,stationwagen,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,A,6.0,6.0,20210507
2021-05-8160,WJ89JD,WJ-89-JD,Aanhangwagen (Caravan),Aanhangwagen,HOME-CAR,36,20210304,NaN,caravan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-8161,WDVN48,WD-VN-48,Aanhangwagen,Aanhangwagen,SARIS,PK,20201222,NaN,gesloten opbouw,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(182, 571)


In [15]:
# Merge closed source data with registrations

# make fields lowercase and add "ovi_"
rdw_closed.index.name='kenteken'
rdw_closed.columns = ['ovi' + re.sub(r'([A-Z])',r'_\1', c).lower() for c in rdw_closed.columns]

# Basic operations
rdw_closed['ovi_private_owners'] = rdw_closed.ovi_eigenaren.str.split('/').apply(lambda x:int(x[0].strip()))
rdw_closed['ovi_company_owner'] = rdw_closed.ovi_eigenaren.str.split('/').apply(lambda x:int(x[1].strip()))
rdw_closed['ovi_owners'] = rdw_closed['ovi_private_owners'] + rdw_closed['ovi_company_owner']
rdw_closed['ovi_under_survey'] = rdw_closed.ovi_wachten_op_keuring.apply(lambda x: {'Ja': True, 'Nee': False}[x])

df = df.merge(rdw_closed, how='left', left_on='kenteken', right_index=True)

### Merge rdw and drz

In [16]:
# merge auction results and rdw queries. Add prefix "rdw_"
rdw = pd.concat([drz, df.add_prefix('rdw_')], axis='columns', sort=False)
# There should be no duplicates in column names
assert all(rdw.columns.value_counts() == 1)
# indices should match
assert rdw.index.isin(drz.index).all() & drz.index.isin(rdw.index).all()

out = rdw.copy()

# Saving

In [17]:
if OPBOD:
    file_name = f'../../../python-nb/data/rdw-data-opbod-{DATE}.pkl'
else:
    file_name = f'../data/rdw-data-{DATE}.pkl'

if NO_PRICE:
    file_name = file_name.replace('.pkl', '-without-price.pkl')
    
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    print(file_name)
    out.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

../data/rdw-data-2021-05.pkl


# Next: download images (or parallel)

Because images might be taken down from the drz site, it is advisable to run the notebook that downloads images soon.